In [158]:
import os
import json
import pandas as pd
from collections import defaultdict

directory = os.fsencode('data/wtt_matches')

In [159]:
tf = pd.read_csv('data/tournaments_wtt.tsv', sep='\t', parse_dates=['StartDateTime', 'EndDateTime'])
tf.sort_values(by=['StartDateTime'], inplace=True)

In [160]:
players = []
genders = defaultdict(lambda: None)

for row in tf.itertuples():
    evt = os.path.join(f'data/wtt_matches', str(row.EventId))
    if not os.path.isdir(evt):
        continue

    for filename in os.listdir(evt):
        if not filename.endswith(".json"):
            continue

        m = None
        with open(os.path.join(f'data/wtt_matches', str(row.EventId), filename)) as f:
            m = json.load(f)

        gender = m['documentCode'][3]

        isTeam = False
        for c in m['competitiors']:
            if isTeam:
                break

            for p in c['players']:
                p['gender'] = gender
                pid = int(p['playerId'])
                if pid > 100000000 or pid < 100000:
                    isTeam = True
                if isTeam:
                    break

                if not p['playerOrgCode']:
                    p['playerOrgCode'] = c['competitiorOrg']
                players.append(p)

        if isTeam and m['teamParentData']:
            for _m in m['teamParentData']['extended_info']['matches']:
                if not _m.get('match_result'):
                    continue

                mm = _m['match_result']
                for c in mm['competitiors']:
                    for p in c['players']:
                        p['gender'] = gender
                        pid = int(p['playerId'])
                        if pid > 100000000 or pid < 100000:
                            continue

                        if not p['playerOrgCode']:
                            p['playerOrgCode'] = c['competitiorOrg']
                        players.append(p)


In [161]:
pf = pd.DataFrame(players)
pf.drop(columns=['playerPosition'], inplace=True)
pf

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
0,112074,FALCK Mattias,None,None,SWE,M
1,104379,KARLSSON Kristian,None,None,SWE,M
2,118994,JEON Jihee,Jihee,JEON,KOR,W
3,121706,Mariia TAILAKOVA,Mariia,TAILAKOVA,RUS,W
4,101648,CHEN Chien-An,None,None,TPE,M
...,...,...,...,...,...,...
53713,107028,Dimitrij Ovtcharov,None,None,GER,M
53714,132992,Alexis Lebrun,None,None,FRA,M
53715,115641,Hugo Calderano,None,None,BRA,M
53716,108226,Elizabeta Samara,None,None,ROU,W


In [162]:
pf[pf.playerName == 'Lubomir PISTEJ']

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
191,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,X
432,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
497,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
549,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
919,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
1253,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
1606,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,X


In [163]:
pf[pf.playerId.isna()]
# only 2, manually fix

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [164]:
name_conflicts = pf.groupby('playerId')['playerName'].apply(set)
conf = pd.DataFrame(name_conflicts)
conf['cnt'] = name_conflicts.apply(len)
conf[conf.cnt > 1]


,playerName,cnt
playerId,,
100032,"{ABDEL-AZIZ Farah, Farah ABDELAZIZ}",2
100439,"{Ahmed SALEH, SALEH Ahmed}",2
100486,"{ALTO Gaston, Gaston ALTO}",2
100621,"{Tiago Apolonia, APOLONIA Tiago, Tiago APOLONIA}",3
100696,"{Omar ASSAR, Omar Assar, ASSAR Omar}",3
...,...,...
200310,"{ADEGOKE Muizz, ADEGOKE Muizz Olawale}",2
200361,"{RIOS Carlos, RIOS ROMERO Carlos Josue Alberto}",2
200391,"{MENEZES ARAUJO Abimael, MENEZES Abimael}",2


In [165]:
id_conflicts = pf.groupby('playerName')['playerId'].apply(set)
idconf = pd.DataFrame(id_conflicts)
idconf['cnt'] = id_conflicts.apply(len)
idconf[idconf.cnt > 1]
# These actually might be different people... 

,playerId,cnt
playerName,,
KUMAHARA Luca,"{206781, 113418}",2
LEE Daeun,"{132702, 135391}",2
Nikhil KUMAR,"{119463, 116936}",2
Yang WANG,"{109995, 112735}",2


In [166]:
org_conflicts = pf.groupby('playerId')['playerOrgCode'].apply(set)
oconf = pd.DataFrame(org_conflicts)
oconf['cnt'] = org_conflicts.apply(len)
oconf[oconf.cnt > 1]


,playerOrgCode,cnt
playerId,,
100868,"{LUX/SVK, CZE/SVK, SVK/CZE, SVK/FRA, SVK/USA, ...",7
102380,"{ENG/GER, GBR, ENG}",3
103163,"{USA/SUI, USA/ESP, USA}",3
103425,"{CMR, CMR/LUX}",2
103903,"{ROU, ROU/ESP}",2
...,...,...
145649,"{PER, LUX/PER}",2
146048,"{PER, PER/ECU}",2
200322,"{LUX/KAZ, KAZ}",2


In [167]:
cleaned = []
for id, rows in pf.groupby('playerId'):
    clrow = {
        'id': id,
        'org': None,
        'name': '',
        'gender': 'X',
    }
    for row in rows.itertuples():
        if row.gender != 'X':
            clrow['gender'] = row.gender
        if '^' in clrow['name'] or (clrow['name'] != row.playerName and min(sum(1 for c in clrow['name'] if c.isupper()), 4) < min(sum(1 for c in row.playerName if c.isupper()), 4)):
            clrow['name'] = row.playerName
        if not clrow['org']:
            clrow['org'] = row.playerOrgCode
        elif row.playerOrgCode:
            # take shorter country code, don't want doubles codes
            if len(row.playerOrgCode) < len(clrow['org']):
                clrow['org'] = row.playerOrgCode

    cleaned.append(clrow)
cf = pd.DataFrame(cleaned)

In [168]:
pf[pf.playerId.isin(cf[cf.org.isna()].id)]

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [169]:
cf.to_csv('data/wtt_cleaned/players.tsv', index=False, sep='\t')